In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import cv2
from sklearn.metrics import confusion_matrix, roc_curve
import seaborn as sns
import datetime
import pathlib
import io
import os
import re
import string
import time
import random
import gensim.downloader as api
from PIL import Image
import tensorflow_datasets as tfds
import tensorflow_probability as tfp
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import (Dense, Flatten, SimpleRNN, InputLayer, Conv1D, Bidirectional,GRU, LSTM )
from tensorflow.keras.losses import BinaryCrossentropy, CategoricalCrossentropy, SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Accuracy, TopKCategoricalAccuracy, CategoricalAccuracy, SparseTopKCategoricalAccuracy
from google.colab import drive
from google.colab import files


In [2]:
BATCH_SIZE = 64

# Pretrained word2vec

In [3]:
pip install gensim


In [4]:
pip install gensim-downloader


ERROR: Could not find a version that satisfies the requirement gensim-downloader (from versions: none)
ERROR: No matching distribution found for gensim-downloader


In [5]:
import gensim.downloader as api

# Load the pre-trained word2vec model (300-dimensional vectors from Google News)
word2vec = api.load("word2vec-google-news-300")


In [6]:
word2vec.vectors.shape

(3000000, 300)

In [7]:
# List of words in the vocabulary
vocab_words = word2vec.index_to_key
print(vocab_words[:10])  # Print first 10 words


['</s>', 'in', 'for', 'that', 'is', 'on', '##', 'The', 'with', 'said']


In [8]:
len(word2vec['The'])

300

In [9]:
word2vec.most_similar('The')

[('This', 0.7025192975997925),
 ('Tthe', 0.6925674080848694),
 ('That', 0.6314951181411743),
 ('A', 0.625857949256897),
 ('_The', 0.6169413328170776),
 ('Although', 0.6020386815071106),
 ('It', 0.6010685563087463),
 ('However', 0.5998899340629578),
 ('Another', 0.5934819579124451),
 ('which', 0.5933603048324585)]

In [10]:
word2vec.most_similar('Man')

[('Woman', 0.6939345002174377),
 ('Boy', 0.5896925330162048),
 ('Girl', 0.584598958492279),
 ('Suspect', 0.5577560067176819),
 ('Couple', 0.5528685450553894),
 ('man', 0.5316051840782166),
 ('Robber', 0.5315312743186951),
 ('Teenager', 0.5161930322647095),
 ('depicts_Michelangelo_Creation', 0.5148172378540039),
 ('Policeman', 0.5136396884918213)]

In [11]:
word2vec

In [12]:
pretrained_embeddings = []


In [13]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

VOCAB_SIZE = 10000
SEQUENCE_LENGTH = 200
EMBEDDING_DIM = 300

def standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    # Replace any character that is not a letter or number with a space
    output = tf.strings.regex_replace(lowercase, '[^a-zA-Z0-9\s]', ' ')
    return output

vectorize_layer = TextVectorization(
    standardize=standardization,
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=SEQUENCE_LENGTH
)


In [14]:
import tensorflow_datasets as tfds

train_ds, val_ds, test_ds = tfds.load('imdb_reviews', split=['train', 'test[:50%]', 'test[50%:]'], as_supervised=True)

training_dataset = train_ds.map(lambda x, y: x)
vectorize_layer.adapt(training_dataset)


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.VE4GQ8_1.0.0/imdb_reviews-train.tfrecor…

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.VE4GQ8_1.0.0/imdb_reviews-test.tfrecord…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.VE4GQ8_1.0.0/imdb_reviews-unsupervised.…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [15]:
vectorize_layer.get_vocabulary()

['',
 '[UNK]',
 'the',
 'and',
 'a',
 'of',
 'to',
 'is',
 'br',
 'it',
 'in',
 'i',
 'this',
 'that',
 's',
 'was',
 'as',
 'for',
 'with',
 'movie',
 'but',
 'film',
 't',
 'you',
 'on',
 'not',
 'he',
 'are',
 'his',
 'have',
 'be',
 'one',
 'all',
 'at',
 'they',
 'by',
 'an',
 'who',
 'so',
 'from',
 'like',
 'there',
 'her',
 'or',
 'just',
 'about',
 'out',
 'if',
 'has',
 'what',
 'some',
 'good',
 'can',
 'more',
 'she',
 'when',
 'very',
 'up',
 'time',
 'no',
 'even',
 'my',
 'would',
 'which',
 'story',
 'only',
 'really',
 'see',
 'their',
 'had',
 'we',
 'were',
 'me',
 'well',
 'than',
 'much',
 'get',
 'bad',
 'been',
 'people',
 'will',
 'do',
 'other',
 'also',
 'into',
 'first',
 'great',
 'because',
 'how',
 'him',
 'don',
 'most',
 'made',
 'its',
 'then',
 'way',
 'make',
 'them',
 'could',
 'too',
 'movies',
 'any',
 'after',
 'think',
 'characters',
 'character',
 'watch',
 'two',
 'films',
 'seen',
 'many',
 'life',
 'being',
 'plot',
 'acting',
 'never',
 'lov

In [16]:
def first_caps(word):
  return word[0].upper() + word[1:]

In [17]:
for i in range(len(vectorize_layer.get_vocabulary())):

  try:
    pretrained_embeddings.append(word2vec[vectorize_layer.get_vocabulary()[i]])
  except:
    print(vectorize_layer.get_vocabulary()[i])

    try:
      pretrained_embeddings.append(word2vec[first_caps(vectorize_layer.get_vocabulary()[i])])
      print('toupper')
    except:
      print("nosolution")
      pretrained_embeddings.append(np.random.normal(loc=0, scale=1, size=(EMBEDDING_DIM)))


  if i % 1000 == 0:
    print('i is ==', i)



nosolution
i is == 0
[UNK]
nosolution
and
toupper
a
toupper
of
toupper
to
toupper
10
nosolution
20
nosolution
30
nosolution
80
nosolution
i is == 1000
90
nosolution
15
nosolution
50
nosolution
100
nosolution
humour
toupper
70
nosolution
40
nosolution
11
nosolution
12
nosolution
favourite
toupper
theatre
toupper
keaton
toupper
60
nosolution
80s
nosolution
70s
nosolution
000
nosolution
tarzan
toupper
i is == 2000
13
nosolution
welles
toupper
sinatra
toupper
14
nosolution
hitchcock
toupper
columbo
toupper
pacino
toupper
25
nosolution
lugosi
toupper
branagh
toupper
60s
nosolution
1950
nosolution
17
nosolution
sandler
toupper
1980
nosolution
2000
nosolution
2006
nosolution
1970
nosolution
bettie
toupper
stanwyck
toupper
2001
nosolution
i is == 3000
brando
toupper
wwii
nosolution
grey
toupper
matthau
toupper
seagal
toupper
1950s
nosolution
18
nosolution
widmark
toupper
spielberg
toupper
elvira
toupper
streisand
toupper
mst3k
nosolution
karloff
toupper
16
nosolution
flynn
toupper
24
nosoluti

In [18]:
word2vec['the']

array([ 0.08007812,  0.10498047,  0.04980469,  0.0534668 , -0.06738281,
       -0.12060547,  0.03515625, -0.11865234,  0.04394531,  0.03015137,
       -0.05688477, -0.07617188,  0.01287842,  0.04980469, -0.08496094,
       -0.06347656,  0.00628662, -0.04321289,  0.02026367,  0.01330566,
       -0.01953125,  0.09277344, -0.171875  , -0.00131989,  0.06542969,
        0.05834961, -0.08251953,  0.0859375 , -0.00318909,  0.05859375,
       -0.03491211, -0.0123291 , -0.0480957 , -0.00302124,  0.05639648,
        0.01495361, -0.07226562, -0.05224609,  0.09667969,  0.04296875,
       -0.03540039, -0.07324219,  0.03271484, -0.06176758,  0.00787354,
        0.0035553 , -0.00878906,  0.0390625 ,  0.03833008,  0.04443359,
        0.06982422,  0.01263428, -0.00445557, -0.03320312, -0.04272461,
        0.09765625, -0.02160645, -0.0378418 ,  0.01190186, -0.01391602,
       -0.11328125,  0.09326172, -0.03930664, -0.11621094,  0.02331543,
       -0.01599121,  0.02636719,  0.10742188, -0.00466919,  0.09

In [19]:
pretrained_embeddings_array = np.array(pretrained_embeddings)
print(pretrained_embeddings_array.shape)

(10000, 300)


In [24]:
from tensorflow.keras.layers import Embedding, Input, Dropout


EMBEDDING_DIM = 300

model = tf.keras.models.Sequential([
    Input(shape =(SEQUENCE_LENGTH,)),

    Embedding(VOCAB_SIZE,
              EMBEDDING_DIM,
              embeddings_initializer = tf.keras.initializers.Constant(pretrained_embeddings_array),
              trainable=False),
    Conv1D(32,3 , activation = "relu"),
    Flatten(),
    Dense(32, activation = "relu"),
    Dropout(0.5),
    Dense(1, activation = "sigmoid")

])

model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)              │ (None, 200, 300)            │       3,000,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_2 (Conv1D)                    │ (None, 198, 32)             │          28,832 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_2 (Flatten)                  │ (None, 6336)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 32)                  │         202,784 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,231,649 (12.33 MB)

 Trainable params: 231,649 (904.88 KB)

 Non-trainable params: 3,000,000 (11.44 MB)

In [26]:
checkpoint_filepath = "/content/drive/MyDrive/nlp/sentiment_analysis/Conv1D.keras"

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

In [27]:
model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(1e-4),
    metrics=['accuracy']
)

In [29]:

for review, label in train_ds.take(1):
  print(review)
  print(label)

tf.Tensor(b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.", shape=(), dtype=string)
tf.Tensor(0, shape=(), dtype=int64)


In [30]:

def vectorizer(review, label):
  return vectorize_layer(review), label

In [31]:

train_dataset = train_ds.map(vectorizer)
val_dataset = val_ds.map(vectorizer)
# test_dataset = test_ds.map(vectorizer)



In [35]:


train_dataset = train_dataset.batch(64).prefetch(buffer_size = tf.data.AUTOTUNE)
val_dataset = val_dataset.batch(64).prefetch(buffer_size = tf.data.AUTOTUNE)
# test_dataset = test_dataset.prefetch(buffer_size = tf.data

In [38]:
# Define the EarlyStopping callback
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Early stopping
early_stopping_callback = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

# Model checkpoint
model_checkpoint_callback = ModelCheckpoint(
    filepath='best_model.keras',
    save_best_only=True,
    monitor='val_loss'
)

# Fit the model with early stopping
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=15,
    callbacks=[model_checkpoint_callback, early_stopping_callback]
)


Epoch 1/15


ValueError: Exception encountered when calling Sequential.call().

[1mCannot take the length of shape with unknown rank.[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=<unknown>, dtype=int64)
  • training=True
  • mask=None